# Udacity Data Scienctist Nano Degree Project 1

## Project Overview
For [Udacity Data Scientist Nano Degree](https://www.udacity.com/course/data-scientist-nanodegree--nd025) project 1 you pick a dataset and using your dataset you choose 3 questions you aspire to answer from the data.

## Dataset
Dataset from the 'Power Laws: Detecting Anomalies in Usage' competition on [drivendata.org](https://www.drivendata.org/competitions/52/anomaly-detection-electricity/page/102/).   

Commercial buildings waste an estimated 15% to 30% of energy used due to poorly maintained, degraded, and improperly controlled equipment.  The goal is to detect abnormal energy consumption, find potential energy saving opportunities and provide actionable recommendations.  

The dataset includes energy consumption data, information about the buildings, historical weather data and public holiday information.  

## Questions
Needs more refinement.  Just a rough list at this point.  
- How much much energy is being wasted?
- What's the environmental impact?
- What's the financial impact?
- What buildings are prone to wasting engergy?
- Is there a time of day more prone to wasting energy?
- Is there a season/time of year more prone to wasting energy?
- How easy/hard/costly is it to fix energy wasting?

Possible outline:  
- How much energy is being wasted and the impact
- What attributes most to wasting engergy (building, time of day, time of year)
- Recommendations to fix

## Import neccessary libraries

In [2]:
import pandas as pd

## Load the data

In [9]:
df_holidays = pd.read_csv('~/data/DSND-Project-1/power-laws-detecting-anomalies-in-usage-holidays.csv')
df_metadata = pd.read_csv('~/data/DSND-Project-1/power-laws-detecting-anomalies-in-usage-metadata.csv')
df_weather = pd.read_csv('~/data/DSND-Project-1/power-laws-detecting-anomalies-in-usage-weather.csv')
df_energy = pd.read_csv('~/data/DSND-Project-1/train.csv')

## Describe the data

In [11]:
df_holidays.head()

,Date;Holiday;site_id
0,2014-10-28;Ohi Day;334_61
1,2010-05-24;Whit Monday;334_61
2,2016-03-28;Easter Monday;038
3,2012-08-15;Assumption of Mary to Heaven;038
4,2016-08-15;Assumption of Mary to Heaven;334_61


In [12]:
df_metadata.head()

,site_id;meter_id;meter_description;units;surface;activity
0,038;38_9759;other;kWh;;office
1,038;38_9729;other;kWh;;office
2,038;38_9742;other;kWh;;office
3,234_203;863;main meter;Wh;5750.0;office
4,038;38_56030;heating;kWh;;laboratory


In [13]:
df_weather.head()

,Timestamp;Temperature;Distance;site_id
0,2013-12-23T19:00:00-06:00;16.0;10.1258193850649;
1,2013-12-28T18:00:00-06:00;18.0;10.1258193850649;
2,2013-12-29T04:00:00-06:00;19.1;8.992769136308905;
3,2013-12-30T00:30:00-06:00;16.0;10.1258193850649;
4,2014-01-01T03:00:00-06:00;18.0;10.1258193850649;


In [14]:
df_energy.head()

,Unnamed: 0,meter_id,Timestamp,Values
0,2532,2,2015-06-11 00:00:00,2035.0
1,2543,2,2015-06-11 00:15:00,2074.0
2,2544,2,2015-06-11 00:30:00,2062.0
3,2525,2,2015-06-11 00:45:00,2025.0
4,2534,2,2015-06-11 01:00:00,2034.0
